# Еще раз про функции в Python

Простейший пример функции:

In [2]:
def add(x, y):
    print(f"Я сложила {x} и {y}")
    return x + y

print(add(2, 3))
print(add(20, 30))

Я сложила 2 и 3
5
Я сложила 20 и 30
50


* `add` - имя функции
* `x, y` - формальные параметры. Переменные, в которых будут храниться значения, с которыми вызвали функцию
* `2, 3` - фактические параметры. То, с чем реально вызвали функцию.
* `20, 30` - тоже фактические параметры
* `5, 50` - это возвращаемые значения, то, что функция сообщает тому, кто ее вызвал
* Печать фразы "я сложила..." - это подочный эффект функции. Т.е. какие-то действия, которые видны тому, кто функцию вызвал. Действия, которые меняют "внешнее состояние".

Другой пример побочного эффекта - это изменение глобальных переменных. Если хочется изменяьть переменную, определенной вне фунцкии, нужно написать команду

`global имя_переменной`:

In [6]:
hello_counter = 0

def hello(name):
    global hello_counter
    hello_counter += 1
    print(f"Hello, {name}!")
    
hello("Ilya")
hello("ILYA")

print(f"Функцию hello была вызвана {hello_counter} раз(а).")

Hello, Ilya!
Hello, ILYA!
Функцию hello была вызвана 2 раз(а).


Нужно стараться избегать использования глобальных переменных, тем более, изменения глобальных переменных, поэтому что это усложняет отладку программы.
Последний пример побочного эффекта - это то, с чем мы уже знакомы, это то, что функция может изменить переданный ей аргумент:

In [7]:
def append42(l):
    l.append(42)
    
l = [10, 20, 30]
append42(l)
print(l)

[10, 20, 30, 42]


Здесь побочный эффект - это то, что изменилось значение переданного аргумента. Т.е. 
изменился список `l`. А возвращаемого значения у функции нет, потому что нет `return`. 

В Python, на самом деле, возвращаемое значение есть всегда, но если вы его явно не указали с помощью `return`, то получается значение `None`

In [8]:
print(append42([]))

None


При отладке своей функции, если вы видите, что она вернула `None`, скоре всего это значит, что вы забыли сделать `return`.

**Старайтесь писать функции без побочных эффектов**, это не всегда возможно, потому что вы все равно должны, например, распечатать что-то на экране или записать в файл. Поэтому старайтесь минимизировать количество функций с побочным эффектом. И если они все-таки есть, то **старайтесь не возвращать значений из функций с побочными эффектами**. Это нужно, чтобы вызов функций выглядел как команда. Например, `append42(l)`. Это помогает различать функции с побочными эффектами и без во время чтения кода.

Приведенные выше правила можно нарушать, если есть на это основания. И на самом деле они часто нарушаются в стандартной библиотеке (какие функции, например, нарушают?)

In [9]:
## Области видимости переменных

In [11]:
x = 10 # это глобальная переменная. Глобальные переменные определены вне функций
y = 20
z = 100

def f():
    print(f"in f(): z = {z}") # мы можем ссылаться на глобальную переменную,
                              # если сами ей ничего не присваиваем. (!!!)
    
    x = 20 # локальная переменная, не имеет отношения к глобальной
    print(f"in f(): x = {x}")
    global y
    y = 30
    print(f"in f(): y = {y}")

print(f"in global before f(): x = {x}, y = {y}") # до вызова функции
f()
print(f"in global after f(): x = {x}, y = {y}") # изменился только y

in global before f(): x = 10, y = 20
in f(): z = 100
in f(): x = 20
in f(): y = 30
in global after f(): x = 10, y = 30


In [12]:
def f():
    x = 42
    
    def g():
        global x
        print(f"inside g(): x = {x}") # эта та x, которая была определена вне функций
        x = 20
        print(f"inside g(): x = {x}")
    
    print(f"inside f() before g(): x = {x}") # это локальный x, который 42
    g();
    print(f"inside f() after g(): x = {x}")

f()
print(f"after f(): x = {x}")

inside f() before g(): x = 42
inside g(): x = 10
inside g(): x = 20
inside f() after g(): x = 42
after f(): x = 20


Итого. Есть глобальная область, вне всех функций. И локальная область каждой функции. Допустим, функция `f` определена в глобальной области, а функция `g` внутри функции `f`. Правила видимости переменных:
1. Можно читать значения переменных из внешних областей. Т.е. `g` может напрямую читать содержимое переменных в f и в глобальной области.
2. Если внутри функции, в любом месте переменной присваивается значение, то переменная с этим именем из внешней области становится недоступной. Например, если в `g` написано `x = 20`, то внешние переменные `x` становятся недоступны.
3. Можно использовать ключевое слово `global`, чтобы начать пользоваться переменной из глобальной области. Старайтесь избегать использования `global`, этот совет аналогичен тому, что нужно стараться избегать функций с побочными эффектами.
4. Изменить переменную из внешней области, например, изменить переменную в `f` из функции `g` возможно с помощью ключевого слова `nonlocal`, похожего по поведению на `global`, но считайте, что я этого не говорил, избегайте изменения переменных во внешних областях.